In [2]:
import torch
import pykeen
from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset
from pykeen.evaluation import RankBasedEvaluator

In [3]:
torch.backends.mps.is_available()

True

In [4]:
mps_device = torch.device("mps")
x = torch.ones(1, device=mps_device)
x

tensor([1.], device='mps:0')

In [5]:
best_model = torch.load('results/baseline/transe_hetionet_best_hpo/trained_model.pkl', mps_device)

In [6]:
dataset = get_dataset(dataset="hetionet")

In [7]:
triples_factory = dataset.training
test_triples = dataset.testing.mapped_triples
ctd_id = torch.as_tensor(triples_factory.relations_to_ids(["CtD"]))
ctd_triples = test_triples[test_triples[:, 1] == ctd_id]

In [8]:
test_triples.shape

torch.Size([225020, 3])

In [ ]:
evaluator = RankBasedEvaluator()
metrics = evaluator.evaluate(best_model, mapped_triples=test_triples, device=mps_device)

The filtered setting was enabled, but there were no `additional_filter_triples`
given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]



Evaluating on mps:0:   0%|          | 0.00/225k [00:00<?, ?triple/s]

Encountered tensors on device_types={'mps'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).


In [9]:
ctd_only_model = torch.load('results/baseline/transe_hetionet_only_ctd_relation/trained_model.pkl', mps_device)

In [10]:
evaluator = RankBasedEvaluator()
metrics = evaluator.evaluate(ctd_only_model, mapped_triples=ctd_triples, device=mps_device)

The filtered setting was enabled, but there were no `additional_filter_triples`
given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]



Evaluating on mps:0:   0%|          | 0.00/71.0 [00:00<?, ?triple/s]

Encountered tensors on device_types={'mps'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).


In [11]:
def print_metrics(metric_results):
    print("MRR: {}".format(metric_results.get_metric("inverseharmonicmeanrank")))
    print("Hits@1: {}".format(metric_results.get_metric("hits_at_1")))
    print("Hits@3: {}".format(metric_results.get_metric("hits_at_3")))
    print("Hits@10: {}".format(metric_results.get_metric("hits_at_10")))

In [12]:
print_metrics(metrics)

MRR: 0.11428647488355637
Hits@1: 0.0
Hits@3: 0.11971830985915492
Hits@10: 0.323943661971831


In [13]:
ctd_triples_validation = dataset.validation.mapped_triples[test_triples[:, 1] == ctd_id]
metrics_ctd_validation = evaluator.evaluate(ctd_only_model, mapped_triples=ctd_triples_validation, device=mps_device)

The filtered setting was enabled, but there were no `additional_filter_triples`
given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]



Evaluating on mps:0:   0%|          | 0.00/71.0 [00:00<?, ?triple/s]

Encountered tensors on device_types={'mps'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).


In [14]:
print_metrics(metrics_ctd_validation)

MRR: 0.08169311285018921
Hits@1: 0.02112676056338028
Hits@3: 0.09859154929577464
Hits@10: 0.20422535211267606
